1. Upload charge_light_output.tar.gz
2. Run all
3. If any fits are unsuccessful, mess with initial parameters

#Setup

In [ ]:
!tar -xzf charge_light_output.tar.gz
!mkdir cl_plots

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as tk
import scipy.special as spi
from scipy.optimize import curve_fit

from google.colab import files
import csv

In [ ]:
def skgauss(n,m,a,alpha, mu):
  phi = np.exp(((((n/m)-mu))**2)/2)**-1
  phicdf = 2*(1+spi.erf(alpha*((n/m)-mu)))
  return a*phi*phicdf

In [ ]:
params = {
   'axes.labelsize': 21,
   'font.size': 16,
   'font.family': 'sans-serif',
   'font.serif': 'Arial',
   'legend.fontsize': 18,
   'xtick.labelsize': 18,
   'ytick.labelsize': 18,
   'axes.labelpad': 15,

   'figure.figsize': [10,8], # value in inches based on dpi of monitor
   'figure.dpi': 105.5, # My monitor has a dpi of around 105.5 px/inch

   'axes.grid': True,
   'grid.linestyle': '-',
   'grid.alpha': 0.25,
   'axes.linewidth': 1,
   'figure.constrained_layout.use': True,


   # Using Paul Tol's notes:
   'axes.prop_cycle':
      mpl.cycler(color=['#4477aa', # blue
                        '#ee6677', # red/pink
                        '#228833', # green
                        '#aa3377', # purple
                        '#66ccee', # cyan
                        '#ccbb44', # yellow
                        '#bbbbbb', # grey
                        ]),

      # Pick either the cycler above, or the cycler below:

      # (mpl.cycler(color=['#4477aa', # blue
      #                     '#ee6677', # red/pink
      #                     '#228833', # green
      #                     '#aa3377', # purple
      #                     '#66ccee', # cyan
      #                     '#ccbb44', # yellow
      #                     '#bbbbbb', # grey
      #                     ]) +
      #   mpl.cycler(linestyle=['-', # solid
      #                         '--', # dashed
      #                         ':', # dotted
      #                         '-.', # dash dot
      #                         (0, (3, 1, 1, 1, 1, 1)), # narrow dash dot dot
      #                         (0, (1, 2, 5, 2, 5, 2)), # dash dash dot
      #                         (0, (5, 2.5, 1, 2.5, 1, 2.5)), # dash dot dot
      #                         ])),

   'lines.linewidth': 2.5,

   'image.cmap': 'jet',
}


plt.rcParams.update(params)

In [ ]:
voltages = [550,560,570,580,590,600,610]
charge_data_0 = np.zeros_like(voltages,dtype=float)
light_data_0 = np.zeros_like(voltages,dtype=float)
charge_err_0 = np.zeros_like(voltages,dtype=float)
light_err_0 = np.zeros_like(voltages,dtype=float)
tracked_levels = ['11.50','11.63','11.88','12.13','12.38','12.50','12.63','12.88','13.13','13.38','13.63','13.88','14.00','14.13','14.38','14.63','14.88','15.13','15.38','15.63','15.88','16.13','16.38','16.63','16.88','17.13','17.38','17.63','17.88','18.13','18.38','18.63','18.88','19.13','19.38','19.63']
p_per_e_0 = np.zeros_like(voltages,dtype=float)

In [ ]:
min_id = 313.9
max_id = 319.7

#0%

## 600V

In [ ]:
num = 0
sizes = []
light_dicts = []
all=[]


for i in range(1,5001,1):
  colls_dict = {}
  try:
    infile = open("cl_600_0/" + str(i) + ".csv","r")
  except FileNotFoundError:
    continue
  all = infile.readlines()
  if int(all[0])!=0:
    anode = int(all[0])
    for j in range(1,len(all)):
      ncoll = all[j].split(",")[1]
      id = format(round(float(all[j].split(",")[0]),2),'.2f')
      id = "0" + str(id)
      if min_id <= float(id) <= max_id:
        colls_dict[id]=int(ncoll)
    num += 1
    sizes.append(anode)
    light_dicts.append(colls_dict)
print(str(num) + " data points")

total_nd_light = []
sum = 0
for i in light_dicts:
  for j in i:
      sum+=i[j]
  total_nd_light.append(sum)
  sum=0




### Charge

In [ ]:
n_bins = 40

#set up bins
bins = np.linspace(min(sizes),max(sizes), n_bins+1)
bin_width = (max(sizes)-min(sizes))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

#Plot histogram
entries, bc, p = plt.hist(sizes, bins=bins, density=True, weights = np.ones_like(sizes)/num)
plt.xlabel("Gain")
plt.ylabel("Normalised counts")
plt.title("Gain distribution [Pure CF4, 60 Torr, dV = 600V]")

#Fit, plot skew-Gaussian
parameters, cov_matrix = curve_fit(skgauss, bin_centres, entries, p0=[100000,0.000001,4,0.001])
y=[]
for i in bin_centres:
  y.append(skgauss(i,parameters[0],parameters[1],parameters[2],parameters[3]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted skew gaussian distribution","Simulation"])

#Data mean, SD, SE
ex_d= 0
var_d = 0
for i in range(len(bin_centres)):
  ex_d += bin_centres[i]*entries[i]*bin_width
for i in range(len(bin_centres)):
  var_d += ((bin_centres[i]-ex_d)**2)*entries[i]*bin_width
sd_d = np.sqrt(var_d)
se_d = sd_d/np.sqrt(num)

#Fit mean, SD, SE
ex = 0
var = 0
for i in range(len(bin_centres)):
  ex += bin_centres[i]*y[i]*bin_width
for i in range(len(bin_centres)):
  var += ((bin_centres[i]-ex)**2)*y[i]*bin_width
sd = np.sqrt(var)
se = sd/np.sqrt(num)

charge_data_0[5] = ex
charge_err_0[5] = np.sqrt((se**2)+(se_d**2))
plt.text(max(sizes)-200000,max(entries)/1.5,"mean = " + str(int(round(ex,0))) + "\nSE = " + str(int(round(np.sqrt((se**2)+(se_d**2)),0))))




plt.savefig("cl_plots/c_600_0.png")

### Light

In [ ]:
n_bins = 40

#set up bins
bins = np.linspace(min(total_nd_light),max(total_nd_light), n_bins+1)
bin_width = (max(total_nd_light)-min(total_nd_light))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

#Plot total histogram
entries, bc, p = plt.hist(total_nd_light, bins=bins, density=True, weights = np.ones_like(total_nd_light)/num)
plt.xlabel("Light production (Arb. units)")
plt.ylabel("Normalised counts")
plt.title("Light distribution [Pure CF4, 60 Torr, dV = 600V]")

#Fit, plot skew-Gaussian
parameters, cov_matrix = curve_fit(skgauss, bin_centres, entries, p0=[5000000,0.000001,4,0.001])
y=[]
for i in bin_centres:
  y.append(skgauss(i,parameters[0],parameters[1],parameters[2],parameters[3]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted skew gaussian distribution","Simulation (CF4 light)"])

#Data mean, SD, SE
ex_d= 0
var_d = 0
for i in range(len(bin_centres)):
  ex_d += bin_centres[i]*entries[i]*bin_width
for i in range(len(bin_centres)):
  var_d += ((bin_centres[i]-ex_d)**2)*entries[i]*bin_width
sd_d = np.sqrt(var_d)
se_d = sd_d/np.sqrt(num)

#Fit mean, SD, SE
ex = 0
var = 0
for i in range(len(bin_centres)):
  ex += bin_centres[i]*y[i]*bin_width
for i in range(len(bin_centres)):
  var += ((bin_centres[i]-ex)**2)*y[i]*bin_width
sd = np.sqrt(var)
se = sd/np.sqrt(num)
plt.text(max(total_nd_light)-200000,max(entries)/1.5,"mean = " + str(int(round(ex,0))) + "\nSE = " + str(int(round(np.sqrt((se**2)+(se_d**2)),0))))

light_data_0[5] = ex
light_err_0[5] = np.sqrt((se**2)+(se_d**2))

plt.savefig("cl_plots/l_600_0.png")

## 550V

In [ ]:
num = 0
sizes = []
light_dicts = []


for i in range(1,5001,1):
  colls_dict = {}
  try:
    infile = open("cl_550_0/" + str(i) + ".csv","r")
  except FileNotFoundError:
    continue
  all = infile.readlines()
  #remove zero and extreme outlier present in this data
  if int(all[0])!=0 and int(all[0])<=150000:
    anode = int(all[0])
    for j in range(1,len(all)):
      ncoll = all[j].split(",")[1]
      id = format(round(float(all[j].split(",")[0]),2),'.2f')
      id = "0" + str(id)
      if min_id <= float(id) <= max_id:
        colls_dict[id]=int(ncoll)
    num += 1
    sizes.append(anode)
    light_dicts.append(colls_dict)
print(str(num) + " data points")

total_nd_light = []
sum = 0
for i in light_dicts:
  for j in i:
      sum+=i[j]
  total_nd_light.append(sum)
  sum=0




### Charge

In [ ]:
n_bins = 40



#set up bins
bins = np.linspace(min(sizes),max(sizes), n_bins+1)
bin_width = (max(sizes)-min(sizes))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

#Plot histogram
entries, bc, p = plt.hist(sizes, bins=bins, density=True, weights = np.ones_like(sizes)/num)
plt.xlabel("Gain")
plt.ylabel("Normalised counts")
plt.title("Gain distribution [Pure CF4, 60 Torr, dV = 550V]")

#Fit, plot skew-Gaussian
parameters, cov_matrix = curve_fit(skgauss, bin_centres, entries, p0=[100000,0.000001,4,0.001])
y=[]
for i in bin_centres:
  y.append(skgauss(i,parameters[0],parameters[1],parameters[2],parameters[3]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted skew gaussian distribution","Simulation"])

#Data mean, SD, SE
ex_d= 0
var_d = 0
for i in range(len(bin_centres)):
  ex_d += bin_centres[i]*entries[i]*bin_width
for i in range(len(bin_centres)):
  var_d += ((bin_centres[i]-ex_d)**2)*entries[i]*bin_width
sd_d = np.sqrt(var_d)
se_d = sd_d/np.sqrt(num)

#Fit mean, SD, SE
ex = 0
var = 0
for i in range(len(bin_centres)):
  ex += bin_centres[i]*y[i]*bin_width
for i in range(len(bin_centres)):
  var += ((bin_centres[i]-ex)**2)*y[i]*bin_width
sd = np.sqrt(var)
se = sd/np.sqrt(num)
plt.text(max(sizes)-20000,max(entries)/1.5,"mean = " + str(int(round(ex,0))) + "\nSE = " + str(int(round(np.sqrt((se**2)+(se_d**2)),0))))

charge_data_0[0] = ex
charge_err_0[0] = np.sqrt((se**2)+(se_d**2))


plt.savefig("cl_plots/c_550_0.png")

### Light

In [ ]:
n_bins = 40

#set up bins
bins = np.linspace(min(total_nd_light),max(total_nd_light), n_bins+1)
bin_width = (max(total_nd_light)-min(total_nd_light))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

#Plot total histogram
entries, bc, p = plt.hist(total_nd_light, bins=bins, density=True, weights = np.ones_like(total_nd_light)/num)
plt.xlabel("Light production (Arb. units)")
plt.ylabel("Normalised counts")
plt.title("Light distribution [Pure CF4, 60 Torr, dV = 550V]")

#Fit, plot skew-Gaussian
parameters, cov_matrix = curve_fit(skgauss, bin_centres, entries, p0=[10000000,0.000001,4,0.001])
y=[]
for i in bin_centres:
  y.append(skgauss(i,parameters[0],parameters[1],parameters[2],parameters[3]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted skew gaussian distribution","Simulation (CF4 light)"])

#Data mean, SD, SE
ex_d= 0
var_d = 0
for i in range(len(bin_centres)):
  ex_d += bin_centres[i]*entries[i]*bin_width
for i in range(len(bin_centres)):
  var_d += ((bin_centres[i]-ex_d)**2)*entries[i]*bin_width
sd_d = np.sqrt(var_d)
se_d = sd_d/np.sqrt(num)

#Fit mean, SD, SE
ex = 0
var = 0
for i in range(len(bin_centres)):
  ex += bin_centres[i]*y[i]*bin_width
for i in range(len(bin_centres)):
  var += ((bin_centres[i]-ex)**2)*y[i]*bin_width
sd = np.sqrt(var)
se = sd/np.sqrt(num)
plt.text(max(total_nd_light)-200000,max(entries)/1.5,"mean = " + str(int(round(ex,0))) + "\nSE = " + str(int(round(np.sqrt((se**2)+(se_d**2)),0))))

light_data_0[0] = ex
light_err_0[0] = np.sqrt((se**2)+(se_d**2))

plt.savefig("cl_plots/l_550_0.png")

## 560V

In [ ]:
num = 0
sizes = []
light_dicts = []


for i in range(1,5001,1):
  colls_dict = {}
  try:
    infile = open("cl_560_0/" + str(i) + ".csv","r")
  except FileNotFoundError:
    continue
  all = infile.readlines()
  if int(all[0])!=0:
    anode = int(all[0])
    for j in range(1,len(all)):
      ncoll = all[j].split(",")[1]
      id = format(round(float(all[j].split(",")[0]),2),'.2f')
      id = "0" + str(id)
      if min_id <= float(id) <= max_id:
        colls_dict[id]=int(ncoll)
    num += 1
    sizes.append(anode)
    light_dicts.append(colls_dict)
print(str(num) + " data points")

total_nd_light = []
sum = 0
for i in light_dicts:
  for j in i:
      sum+=i[j]
  total_nd_light.append(sum)
  sum=0




### Charge

In [ ]:
n_bins = 30

#set up bins
bins = np.linspace(min(sizes),max(sizes), n_bins+1)
bin_width = (max(sizes)-min(sizes))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

#Plot histogram
entries, bc, p = plt.hist(sizes, bins=bins, density=True, weights = np.ones_like(sizes)/num)
plt.xlabel("Gain")
plt.ylabel("Normalised counts")
plt.title("Gain distribution [Pure CF4, 60 Torr, dV = 560V]")

#Fit, plot skew-Gaussian
parameters, cov_matrix = curve_fit(skgauss, bin_centres, entries, p0=[100000,0.000001,4,0.001])
y=[]
for i in bin_centres:
  y.append(skgauss(i,parameters[0],parameters[1],parameters[2],parameters[3]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted skew gaussian distribution","Simulation"])

#Data mean, SD, SE
ex_d= 0
var_d = 0
for i in range(len(bin_centres)):
  ex_d += bin_centres[i]*entries[i]*bin_width
for i in range(len(bin_centres)):
  var_d += ((bin_centres[i]-ex_d)**2)*entries[i]*bin_width
sd_d = np.sqrt(var_d)
se_d = sd_d/np.sqrt(num)

#Fit mean, SD, SE
ex = 0
var = 0
for i in range(len(bin_centres)):
  ex += bin_centres[i]*y[i]*bin_width
for i in range(len(bin_centres)):
  var += ((bin_centres[i]-ex)**2)*y[i]*bin_width
sd = np.sqrt(var)
se = sd/np.sqrt(num)
plt.text(max(sizes)-20000,max(entries)/1.5,"mean = " + str(int(round(ex,0))) + "\nSE = " + str(int(round(np.sqrt((se**2)+(se_d**2)),0))))

charge_data_0[1] = ex
charge_err_0[1] = np.sqrt((se**2)+(se_d**2))


plt.savefig("cl_plots/c_560_0.png")

### Light

In [ ]:
n_bins = 30

#set up bins
bins = np.linspace(min(total_nd_light),max(total_nd_light), n_bins+1)
bin_width = (max(total_nd_light)-min(total_nd_light))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

#Plot total histogram
entries, bc, p = plt.hist(total_nd_light, bins=bins, density=True, weights = np.ones_like(total_nd_light)/num)
plt.xlabel("Light production (Arb. units)")
plt.ylabel("Normalised counts")
plt.title("Light distribution [Pure CF4, 60 Torr, dV = 560V]")

#Fit, plot skew-Gaussian
parameters, cov_matrix = curve_fit(skgauss, bin_centres, entries, p0=[3000000,0.00001,4,1])
y=[]
for i in bin_centres:
  y.append(skgauss(i,parameters[0],parameters[1],parameters[2],parameters[3]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted skew gaussian distribution","Simulation (CF4 light)"])

#Data mean, SD, SE
ex_d= 0
var_d = 0
for i in range(len(bin_centres)):
  ex_d += bin_centres[i]*entries[i]*bin_width
for i in range(len(bin_centres)):
  var_d += ((bin_centres[i]-ex_d)**2)*entries[i]*bin_width
sd_d = np.sqrt(var_d)
se_d = sd_d/np.sqrt(num)

#Fit mean, SD, SE
ex = 0
var = 0
for i in range(len(bin_centres)):
  ex += bin_centres[i]*y[i]*bin_width
for i in range(len(bin_centres)):
  var += ((bin_centres[i]-ex)**2)*y[i]*bin_width
sd = np.sqrt(var)
se = sd/np.sqrt(num)
light_data_0[1] = ex
light_err_0[1] = np.sqrt((se**2)+(se_d**2))
plt.text(max(total_nd_light)-200000,max(entries)/1.5,"mean = " + str(int(round(ex,0))) + "\nSE = " + str(int(round(np.sqrt((se**2)+(se_d**2)),0))))



plt.savefig("cl_plots/l_560_0.png")

## 570V

In [ ]:
num = 0
sizes = []
light_dicts = []


for i in range(1,5001,1):
  colls_dict = {}
  try:
    infile = open("cl_570_0/" + str(i) + ".csv","r")
  except FileNotFoundError:
    continue
  all = infile.readlines()
  if int(all[0])!=0:
    anode = int(all[0])
    for j in range(1,len(all)):
      ncoll = all[j].split(",")[1]
      id = format(round(float(all[j].split(",")[0]),2),'.2f')
      id = "0" + str(id)
      if min_id <= float(id) <= max_id:
        colls_dict[id]=int(ncoll)
    num += 1
    sizes.append(anode)
    light_dicts.append(colls_dict)
print(str(num) + " data points")

total_nd_light = []
sum = 0
for i in light_dicts:
  for j in i:
      sum+=i[j]
  total_nd_light.append(sum)
  sum=0




### Charge

In [ ]:
n_bins = 30

#set up bins
bins = np.linspace(min(sizes),max(sizes), n_bins+1)
bin_width = (max(sizes)-min(sizes))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

#Plot histogram
entries, bc, p = plt.hist(sizes, bins=bins, density=True, weights = np.ones_like(sizes)/num)
plt.xlabel("Gain")
plt.ylabel("Normalised counts")
plt.title("Gain distribution [Pure CF4, 60 Torr, dV = 570V]")

#Fit, plot skew-Gaussian
parameters, cov_matrix = curve_fit(skgauss, bin_centres, entries, p0=[20000,0.000001,5,0.1])
y=[]
for i in bin_centres:
  y.append(skgauss(i,parameters[0],parameters[1],parameters[2],parameters[3]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted skew gaussian distribution","Simulation"])

#Data mean, SD, SE
ex_d= 0
var_d = 0
for i in range(len(bin_centres)):
  ex_d += bin_centres[i]*entries[i]*bin_width
for i in range(len(bin_centres)):
  var_d += ((bin_centres[i]-ex_d)**2)*entries[i]*bin_width
sd_d = np.sqrt(var_d)
se_d = sd_d/np.sqrt(num)

#Fit mean, SD, SE
ex = 0
var = 0
for i in range(len(bin_centres)):
  ex += bin_centres[i]*y[i]*bin_width
for i in range(len(bin_centres)):
  var += ((bin_centres[i]-ex)**2)*y[i]*bin_width
sd = np.sqrt(var)
se = sd/np.sqrt(num)
plt.text(max(sizes)-20000,max(entries)/1.5,"mean = " + str(int(round(ex,0))) + "\nSE = " + str(int(round(np.sqrt((se**2)+(se_d**2)),0))))

charge_data_0[2] = ex
charge_err_0[2] = np.sqrt((se**2)+(se_d**2))


plt.savefig("cl_plots/c_570_0.png")

### Light

In [ ]:
n_bins = 30

#set up bins
bins = np.linspace(min(total_nd_light),max(total_nd_light), n_bins+1)
bin_width = (max(total_nd_light)-min(total_nd_light))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

#Plot total histogram
entries, bc, p = plt.hist(total_nd_light, bins=bins, density=True, weights = np.ones_like(total_nd_light)/num)
plt.xlabel("Light production (Arb. units)")
plt.ylabel("Normalised counts")
plt.title("Light distribution [Pure CF4, 60 Torr, dV = 570V]")

#Fit, plot skew-Gaussian
parameters, cov_matrix = curve_fit(skgauss, bin_centres, entries, p0=[200000,0.00001,8,0.1])
y=[]
for i in bin_centres:
  y.append(skgauss(i,parameters[0],parameters[1],parameters[2],parameters[3]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted skew gaussian distribution","Simulation (CF4 light)"])

#Data mean, SD, SE
ex_d= 0
var_d = 0
for i in range(len(bin_centres)):
  ex_d += bin_centres[i]*entries[i]*bin_width
for i in range(len(bin_centres)):
  var_d += ((bin_centres[i]-ex_d)**2)*entries[i]*bin_width
sd_d = np.sqrt(var_d)
se_d = sd_d/np.sqrt(num)

#Fit mean, SD, SE
ex = 0
var = 0
for i in range(len(bin_centres)):
  ex += bin_centres[i]*y[i]*bin_width
for i in range(len(bin_centres)):
  var += ((bin_centres[i]-ex)**2)*y[i]*bin_width
sd = np.sqrt(var)
se = sd/np.sqrt(num)
light_data_0[2] = ex
light_err_0[2] = np.sqrt((se**2)+(se_d**2))
plt.text(max(total_nd_light)-200000,max(entries)/1.5,"mean = " + str(int(round(ex,0))) + "\nSE = " + str(int(round(np.sqrt((se**2)+(se_d**2)),0))))

print(parameters)

plt.savefig("cl_plots/l_570_0.png")

## 580V

In [ ]:
num = 0
sizes = []
light_dicts = []


for i in range(1,5001,1):
  colls_dict = {}
  try:
    infile = open("cl_580_0/" + str(i) + ".csv","r")
  except FileNotFoundError:
    continue
  all = infile.readlines()
  if int(all[0])!=0:
    anode = int(all[0])
    for j in range(1,len(all)):
      ncoll = all[j].split(",")[1]
      id = format(round(float(all[j].split(",")[0]),2),'.2f')
      id = "0" + str(id)
      if min_id <= float(id) <= max_id:
        colls_dict[id]=int(ncoll)
    num += 1
    sizes.append(anode)
    light_dicts.append(colls_dict)
print(str(num) + " data points")

total_nd_light = []
sum = 0
for i in light_dicts:
  for j in i:
      sum+=i[j]
  total_nd_light.append(sum)
  sum=0




### Charge

In [ ]:
n_bins = 30

#set up bins
bins = np.linspace(min(sizes),max(sizes), n_bins+1)
bin_width = (max(sizes)-min(sizes))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

#Plot histogram
entries, bc, p = plt.hist(sizes, bins=bins, density=True, weights = np.ones_like(sizes)/num)
plt.xlabel("Gain")
plt.ylabel("Normalised counts")
plt.title("Gain distribution [Pure CF4, 60 Torr, dV = 580V]")

#Fit, plot skew-Gaussian
parameters, cov_matrix = curve_fit(skgauss, bin_centres, entries, p0=[30000,0.000001,6,0.1])
y=[]
for i in bin_centres:
  y.append(skgauss(i,parameters[0],parameters[1],parameters[2],parameters[3]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted skew gaussian distribution","Simulation"])

#Data mean, SD, SE
ex_d= 0
var_d = 0
for i in range(len(bin_centres)):
  ex_d += bin_centres[i]*entries[i]*bin_width
for i in range(len(bin_centres)):
  var_d += ((bin_centres[i]-ex_d)**2)*entries[i]*bin_width
sd_d = np.sqrt(var_d)
se_d = sd_d/np.sqrt(num)

#Fit mean, SD, SE
ex = 0
var = 0
for i in range(len(bin_centres)):
  ex += bin_centres[i]*y[i]*bin_width
for i in range(len(bin_centres)):
  var += ((bin_centres[i]-ex)**2)*y[i]*bin_width
sd = np.sqrt(var)
se = sd/np.sqrt(num)
plt.text(max(sizes)-20000,max(entries)/1.5,"mean = " + str(int(round(ex,0))) + "\nSE = " + str(int(round(np.sqrt((se**2)+(se_d**2)),0))))

charge_data_0[3] = ex
charge_err_0[3] = np.sqrt((se**2)+(se_d**2))


plt.savefig("cl_plots/c_580_0.png")

### Light

In [ ]:
n_bins = 30

#set up bins
bins = np.linspace(min(total_nd_light),max(total_nd_light), n_bins+1)
bin_width = (max(total_nd_light)-min(total_nd_light))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

#Plot total histogram
entries, bc, p = plt.hist(total_nd_light, bins=bins, density=True, weights = np.ones_like(total_nd_light)/num)
plt.xlabel("Light production (Arb. units)")
plt.ylabel("Normalised counts")
plt.title("Light distribution [Pure CF4, 60 Torr, dV = 580V]")

#Fit, plot skew-Gaussian
parameters, cov_matrix = curve_fit(skgauss, bin_centres, entries, p0=[200000,0.00001,8,0.1])
y=[]
for i in bin_centres:
  y.append(skgauss(i,parameters[0],parameters[1],parameters[2],parameters[3]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted skew gaussian distribution","Simulation (CF4 light)"])

#Data mean, SD, SE
ex_d= 0
var_d = 0
for i in range(len(bin_centres)):
  ex_d += bin_centres[i]*entries[i]*bin_width
for i in range(len(bin_centres)):
  var_d += ((bin_centres[i]-ex_d)**2)*entries[i]*bin_width
sd_d = np.sqrt(var_d)
se_d = sd_d/np.sqrt(num)

#Fit mean, SD, SE
ex = 0
var = 0
for i in range(len(bin_centres)):
  ex += bin_centres[i]*y[i]*bin_width
for i in range(len(bin_centres)):
  var += ((bin_centres[i]-ex)**2)*y[i]*bin_width
sd = np.sqrt(var)
se = sd/np.sqrt(num)
light_data_0[3] = ex
light_err_0[3] = np.sqrt((se**2)+(se_d**2))
plt.text(max(total_nd_light)-200000,max(entries)/1.5,"mean = " + str(int(round(ex,0))) + "\nSE = " + str(int(round(np.sqrt((se**2)+(se_d**2)),0))))

plt.savefig("cl_plots/l_580_0.png")

## 590V

In [ ]:
num = 0
sizes = []
light_dicts = []


for i in range(1,5001,1):
  colls_dict = {}
  try:
    infile = open("cl_590_0/" + str(i) + ".csv","r")
  except FileNotFoundError:
    continue
  all = infile.readlines()
  if int(all[0])!=0:
    anode = int(all[0])
    for j in range(1,len(all)):
      ncoll = all[j].split(",")[1]
      id = format(round(float(all[j].split(",")[0]),2),'.2f')
      id = "0" + str(id)
      if min_id <= float(id) <= max_id:
        colls_dict[id]=int(ncoll)
    num += 1
    sizes.append(anode)
    light_dicts.append(colls_dict)
print(str(num) + " data points")

total_nd_light = []
sum = 0
for i in light_dicts:
  for j in i:
      sum+=i[j]
  total_nd_light.append(sum)
  sum=0




### Charge

In [ ]:
n_bins = 30

#set up bins
bins = np.linspace(min(sizes),max(sizes), n_bins+1)
bin_width = (max(sizes)-min(sizes))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

#Plot histogram
entries, bc, p = plt.hist(sizes, bins=bins, density=True, weights = np.ones_like(sizes)/num)
plt.xlabel("Gain")
plt.ylabel("Normalised counts")
plt.title("Gain distribution [Pure CF4, 60 Torr, dV = 590V]")

#Fit, plot skew-Gaussian
parameters, cov_matrix = curve_fit(skgauss, bin_centres, entries, p0=[50000,0.000001,6,0.1])
y=[]
for i in bin_centres:
  y.append(skgauss(i,parameters[0],parameters[1],parameters[2],parameters[3]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted skew gaussian distribution","Simulation"])

#Data mean, SD, SE
ex_d= 0
var_d = 0
for i in range(len(bin_centres)):
  ex_d += bin_centres[i]*entries[i]*bin_width
for i in range(len(bin_centres)):
  var_d += ((bin_centres[i]-ex_d)**2)*entries[i]*bin_width
sd_d = np.sqrt(var_d)
se_d = sd_d/np.sqrt(num)

#Fit mean, SD, SE
ex = 0
var = 0
for i in range(len(bin_centres)):
  ex += bin_centres[i]*y[i]*bin_width
for i in range(len(bin_centres)):
  var += ((bin_centres[i]-ex)**2)*y[i]*bin_width
sd = np.sqrt(var)
se = sd/np.sqrt(num)
plt.text(max(sizes)-20000,max(entries)/1.5,"mean = " + str(int(round(ex,0))) + "\nSE = " + str(int(round(np.sqrt((se**2)+(se_d**2)),0))))

charge_data_0[4] = ex
charge_err_0[4] = np.sqrt((se**2)+(se_d**2))


plt.savefig("cl_plots/c_590_0.png")

### Light

In [ ]:
n_bins = 30

#set up bins
bins = np.linspace(min(total_nd_light),max(total_nd_light), n_bins+1)
bin_width = (max(total_nd_light)-min(total_nd_light))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

#Plot total histogram
entries, bc, p = plt.hist(total_nd_light, bins=bins, density=True, weights = np.ones_like(total_nd_light)/num)
plt.xlabel("Light production (Arb. units)")
plt.ylabel("Normalised counts")
plt.title("Light distribution [Pure CF4, 60 Torr, dV = 590V]")

#Fit, plot skew-Gaussian
parameters, cov_matrix = curve_fit(skgauss, bin_centres, entries, p0=[200000,0.00001,8,0.1])
y=[]
for i in bin_centres:
  y.append(skgauss(i,parameters[0],parameters[1],parameters[2],parameters[3]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted skew gaussian distribution","Simulation (CF4 light)"])

#Data mean, SD, SE
ex_d= 0
var_d = 0
for i in range(len(bin_centres)):
  ex_d += bin_centres[i]*entries[i]*bin_width
for i in range(len(bin_centres)):
  var_d += ((bin_centres[i]-ex_d)**2)*entries[i]*bin_width
sd_d = np.sqrt(var_d)
se_d = sd_d/np.sqrt(num)

#Fit mean, SD, SE
ex = 0
var = 0
for i in range(len(bin_centres)):
  ex += bin_centres[i]*y[i]*bin_width
for i in range(len(bin_centres)):
  var += ((bin_centres[i]-ex)**2)*y[i]*bin_width
sd = np.sqrt(var)
se = sd/np.sqrt(num)
light_data_0[4] = ex
light_err_0[4] = np.sqrt((se**2)+(se_d**2))
plt.text(max(total_nd_light)-200000,max(entries)/1.5,"mean = " + str(int(round(ex,0))) + "\nSE = " + str(int(round(np.sqrt((se**2)+(se_d**2)),0))))

plt.savefig("cl_plots/l_590_0.png")

## 610V

In [ ]:
num = 0
sizes = []
light_dicts = []


for i in range(1,5001,1):
  colls_dict = {}
  try:
    infile = open("cl_610_0/" + str(i) + ".csv","r")
  except FileNotFoundError:
    continue
  all = infile.readlines()
  if int(all[0])!=0:
    anode = int(all[0])
    for j in range(1,len(all)):
      ncoll = all[j].split(",")[1]
      id = format(round(float(all[j].split(",")[0]),2),'.2f')
      id = "0" + str(id)
      if min_id <= float(id) <= max_id:
        colls_dict[id]=int(ncoll)
    num += 1
    sizes.append(anode)
    light_dicts.append(colls_dict)
print(str(num) + " data points")

total_nd_light = []
sum = 0
for i in light_dicts:
  for j in i:
      sum+=i[j]
  total_nd_light.append(sum)
  sum=0




### Charge

In [ ]:
n_bins = 30

#set up bins
bins = np.linspace(min(sizes),max(sizes), n_bins+1)
bin_width = (max(sizes)-min(sizes))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

#Plot histogram
entries, bc, p = plt.hist(sizes, bins=bins, density=True, weights = np.ones_like(sizes)/num)
plt.xlabel("Gain")
plt.ylabel("Normalised counts")
plt.title("Gain distribution [Pure CF4, 60 Torr, dV = 610V]")

#Fit, plot skew-Gaussian
parameters, cov_matrix = curve_fit(skgauss, bin_centres, entries, p0=[200000,0.000001,6,0.1])
y=[]
for i in bin_centres:
  y.append(skgauss(i,parameters[0],parameters[1],parameters[2],parameters[3]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted skew gaussian distribution","Simulation"])

#Data mean, SD, SE
ex_d= 0
var_d = 0
for i in range(len(bin_centres)):
  ex_d += bin_centres[i]*entries[i]*bin_width
for i in range(len(bin_centres)):
  var_d += ((bin_centres[i]-ex_d)**2)*entries[i]*bin_width
sd_d = np.sqrt(var_d)
se_d = sd_d/np.sqrt(num)

#Fit mean, SD, SE
ex = 0
var = 0
for i in range(len(bin_centres)):
  ex += bin_centres[i]*y[i]*bin_width
for i in range(len(bin_centres)):
  var += ((bin_centres[i]-ex)**2)*y[i]*bin_width
sd = np.sqrt(var)
se = sd/np.sqrt(num)
plt.text(max(sizes)-20000,max(entries)/1.5,"mean = " + str(int(round(ex,0))) + "\nSE = " + str(int(round(np.sqrt((se**2)+(se_d**2)),0))))

charge_data_0[6] = ex
charge_err_0[6] = np.sqrt((se**2)+(se_d**2))


plt.savefig("cl_plots/c_610_0.png")

### Light

In [ ]:
n_bins = 30

#set up bins
bins = np.linspace(min(total_nd_light),max(total_nd_light), n_bins+1)
bin_width = (max(total_nd_light)-min(total_nd_light))/n_bins
bin_centres = []
for i in range(len(bins)-1):
  bin_centres.append((bins[1]-bins[0])*(i)+(bins[1]+bins[0])/2)

#Plot total histogram
entries, bc, p = plt.hist(total_nd_light, bins=bins, density=True, weights = np.ones_like(total_nd_light)/num)
plt.xlabel("Light production (Arb. units)")
plt.ylabel("Normalised counts")
plt.title("Light distribution [Pure CF4, 60 Torr, dV = 610V]")

#Fit, plot skew-Gaussian
parameters, cov_matrix = curve_fit(skgauss, bin_centres, entries, p0=[1000000,0.00001,8,0.1])
y=[]
for i in bin_centres:
  y.append(skgauss(i,parameters[0],parameters[1],parameters[2],parameters[3]))
plt.plot(bin_centres,y,"o")
plt.legend(["Fitted skew gaussian distribution","Simulation (CF4 light)"])

#Data mean, SD, SE
ex_d= 0
var_d = 0
for i in range(len(bin_centres)):
  ex_d += bin_centres[i]*entries[i]*bin_width
for i in range(len(bin_centres)):
  var_d += ((bin_centres[i]-ex_d)**2)*entries[i]*bin_width
sd_d = np.sqrt(var_d)
se_d = sd_d/np.sqrt(num)

#Fit mean, SD, SE
ex = 0
var = 0
for i in range(len(bin_centres)):
  ex += bin_centres[i]*y[i]*bin_width
for i in range(len(bin_centres)):
  var += ((bin_centres[i]-ex)**2)*y[i]*bin_width
sd = np.sqrt(var)
se = sd/np.sqrt(num)
light_data_0[6] = ex
light_err_0[6] = np.sqrt((se**2)+(se_d**2))
plt.text(max(total_nd_light)-200000,max(entries)/1.5,"mean = " + str(int(round(ex,0))) + "\nSE = " + str(int(round(np.sqrt((se**2)+(se_d**2)),0))))

plt.savefig("cl_plots/l_610_0.png")

# Comparison

In [ ]:
#fraction of collisions that result in scintillation
scintillation_fraction = 1
efficiency = 0.17585
light_data_0 = light_data_0*scintillation_fraction
light_err_0 = light_err_0*scintillation_fraction

p_per_e_0[0] = efficiency*light_data_0[0]/charge_data_0[0]
p_per_e_0[1] = efficiency*light_data_0[1]/charge_data_0[1]
p_per_e_0[2] = efficiency*light_data_0[2]/charge_data_0[2]
p_per_e_0[3] = efficiency*light_data_0[3]/charge_data_0[3]
p_per_e_0[4] = efficiency*light_data_0[4]/charge_data_0[4]
p_per_e_0[5] = efficiency*light_data_0[5]/charge_data_0[5]
p_per_e_0[6] = efficiency*light_data_0[6]/charge_data_0[6]



In [ ]:
ax = plt.axes()

# 0%
ax.scatter(charge_data_0,light_data_0)
ax.errorbar(charge_data_0,light_data_0,xerr = charge_err_0,yerr = light_err_0,ls='none')

ax.set_xlabel("Electrons collected")
ax.set_ylabel("Photons produced")
ax.set_title("[60 Torr, pure CF4]")
#ax.plot(np.linspace(charge_data_0[0],max(charge_data_0)),np.multiply(np.linspace(charge_data_0[0],max(charge_data_0)),207))
#ax.legend(["Simulation data","y=207x"])

In [ ]:
ax = plt.axes()
ax.scatter(charge_data_0,p_per_e_0)
ax.set_xlabel("Electrons collected")
ax.set_ylabel("Photons per electron produced")
ax.set_title("[60 Torr, pure CF4]")